In [1]:
import pandas as pd

## Import new data

In [2]:
new_test = pd.read_table('/media/storage/TG/data_compare/new_data/y/fold_0_test.tsv')
new_train = pd.read_table('/media/storage/TG/data_compare/new_data/y/fold_0_train.tsv')
new_val = pd.read_table('/media/storage/TG/data_compare/new_data/y/fold_0_val.tsv')
new = pd.concat([new_val, new_test, new_train])
new.head()

,IID,ancestry
0,HG00144,GBR
1,HG00264,GBR
2,HG00237,GBR
3,HG00410,CHS
4,HG00097,GBR


In [3]:
new_test_x = pd.read_table('/media/storage/TG/data_compare/new_data/x/fold_0_test_projections.csv.eigenvec.sscore')
new_train_x = pd.read_table('/media/storage/TG/data_compare/new_data/x/fold_0_train_projections.csv.eigenvec.sscore')
new_val_x = pd.read_table('/media/storage/TG/data_compare/new_data/x/fold_0_val_projections.csv.eigenvec.sscore')

## Import old data

In [4]:
old = pd.read_csv('/media/storage/TG/data_compare/old_data/y/y_all.csv')
old.head()

,IID,pop
0,HG00096,GBR
1,HG00097,GBR
2,HG00099,GBR
3,HG00100,GBR
4,HG00101,GBR


## Merge old and new

In [5]:
all = pd.merge(old, new)
all.head()
samples = list(all['IID'].values)
len(samples)

1415

## Compare new and old ancestry

In [6]:
err_samples = all[all['pop'] != all['ancestry']]
err_samples.head()

,IID,pop,ancestry


## Samples that are present in old_data, but not in new_data

In [12]:
new.rename(columns = {'ancestry':'pop'}, inplace = True)
diff = old.merge(new, how = 'outer' ,indicator=True).loc[lambda x : x['_merge']=='left_only']
filtered = new[~new['IID'].isin(samples)]
filtered.to_csv('/media/storage/TG/data_compare/new_data/samples.tsv', sep='\t', index=False)

## Save filtered new_data

In [9]:
save = new_test[new_test['IID'].isin(samples)]
save.to_csv('/media/storage/TG/data_compare/new_data/y_filtered/fold_0_test.tsv', sep='\t', index=False)
save = new_train[new_train['IID'].isin(samples)]
save.to_csv('/media/storage/TG/data_compare/new_data/y_filtered/fold_0_train.tsv', sep='\t', index=False)
save = new_val[new_val['IID'].isin(samples)]
save.to_csv('/media/storage/TG/data_compare/new_data/y_filtered/fold_0_val.tsv', sep='\t', index=False)

In [14]:
save = new_test_x[new_test_x['#IID'].isin(samples)]
save.to_csv('/media/storage/TG/data_compare/new_data/x_filtered/fold_0_test_projections.csv.eigenvec.sscore', sep='\t', index=False)
save = new_train_x[new_train_x['#IID'].isin(samples)]
save.to_csv('/media/storage/TG/data_compare/new_data/x_filtered/fold_0_train_projections.csv.eigenvec.sscore', sep='\t', index=False)
save = new_val_x[new_val_x['#IID'].isin(samples)]
save.to_csv('/media/storage/TG/data_compare/new_data/x_filtered/fold_0_val_projections.csv.eigenvec.sscore', sep='\t', index=False)

## Run experiment with cross validation

In [29]:
%cd ../local
!python experiment.py
%cd ../JupyterNotebooks

/home/genxadmin/test_ukb/src/local
/home/genxadmin/miniconda3/envs/uk/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
{'study': 'tg', 'split_dir': '/media/storage/TG/data_compare', 'node': 'ALL', 'fold_index': 0, 'phenotype': {'name': 'ancestry'}, 'data': {'genotype': {'root': '/media/storage/TG/data/chip/superpop_split/genotypes/${node}/fold_${fold_index}', 'train': '${.root}_train', 'val': '${.root}_val', 'test': '${.root}_test'}, 'x_reduced': {'root': '${split_dir}/new_data/x_filtered/fold_${fold_index}', 'train': '${.root}_train_projections.csv.eigenvec.sscore', 'val': '${.root}_val_projections.csv.eigenvec.sscore', 'test': '${.root}_test_projections.csv.eigenvec.sscore'}, 'phenotype': {'name': 'ancestry', 'root': '${split_dir}/new_data/y_filtered/fold_${fold_index}', 'train': '${.root}

## Results
```
'test_score': array([0.97887324, 0.97183099, 0.98591549, 0.98591549, 0.97887324, 1., 0.9929078, 0.9858156, 0.9929078, 1.]),
'train_score': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])}
```
